# Requests

`requests` is a Python library that allows you to send http requests. Read more about this library in the [official documentation](https://requests.readthedocs.io/en/latest/#).

In the following cell we create container of the `httpbin` which allows to check headers of the requests.

In [1]:
import docker
import requests

docker_client = docker.from_env()

container = docker_client.containers.run(
    image="kennethreitz/httpbin",
    ports={80: 80},
    detach=True,
    remove=True
)

To keep the environment clean, we need to stop the docker containers.

In [ ]:
container.stop()

## Default headers

There are some headers that the `requests` library generates itself - even if you've specified empty headers, there will be some headers in the request.

---

The following example shows that requests with headers specified as empty lead to requests that still have some headers.

By sending the request to the `httpbin` we created earlier, in the `/headers` path, we can check the headers of the request - they'll be returned as a response.

In [43]:

prepared_request = requests.Request(
    'GET', 
    'http://localhost:80/headers', 
    headers={}
).prepare()

session = requests.Session()
response = session.send(prepared_request)

print(response.content.decode("utf-8"))

{
  "headers": {
    "Accept-Encoding": "identity", 
    "Host": "localhost", 
    "User-Agent": "python-urllib3/1.26.5"
  }
}



Even though we explicitly specified `headers={}`, it still results in some filled headers.